# schedule

In [3]:
# -*- conding:utf-8 -*-
import threading
import time
import sys

import schedule


def p1():
    sys.stdout.write('1: ' + time.strftime("%d.%m.%Y %H:%M:%S") + '\n')


def p2():
    sys.stdout.write('2: ' + time.strftime("%d.%m.%Y %H:%M:%S") + '\n')


def loop():
    while True:
        schedule.run_pending()


schedule.every(1).seconds.do(p1)
schedule.every(3).seconds.do(p2)
schedule.every(1).hour.do(p1)
t = threading.Thread(target=loop, name='send_heart')
t.setDaemon(True)
t.start()
input('\n')

2: 20.04.2020 23:33:20
1: 20.04.2020 23:33:20
1: 20.04.2020 23:33:20
1: 20.04.2020 23:33:21
2: 20.04.2020 23:33:22
1: 20.04.2020 23:33:22
1: 20.04.2020 23:33:22
2: 20.04.2020 23:33:23
1: 20.04.2020 23:33:23
2: 20.04.2020 23:33:23
1: 20.04.2020 23:33:23
1: 20.04.2020 23:33:23
2: 20.04.2020 23:33:24
1: 20.04.2020 23:33:24
1: 20.04.2020 23:33:24
1: 20.04.2020 23:33:24
1: 20.04.2020 23:33:24
1: 20.04.2020 23:33:24

 


' '

In [7]:
schedule.every(2).hours.do(p1)

Every 2 hours do p1() (last run: [never], next run: 2020-04-12 01:00:04)

In [1]:
# -*- conding:utf-8 -*-
import threading
import time

import schedule


def p1():
    print(1, time.strftime("%d.%m.%Y %H:%M:%S"))


def p2():
    print(2, time.strftime("%d.%m.%Y %H:%M:%S"))


def loop():
    while True:
        schedule.run_pending()
        # 阻塞调度
        time.sleep(4)


schedule.every(1).seconds.do(p1)
schedule.every(4).seconds.do(p2)
t = threading.Thread(target=loop, name='send_heart')
t.setDaemon(True)
t.start()
input('\n')

1 11.04.2020 11:02:09
2 11.04.2020 11:02:09
1 11.04.2020 11:02:13
2 11.04.2020 11:02:13
1 11.04.2020 11:02:17
2 11.04.2020 11:02:17




''

# APScheduler

<https://blog.csdn.net/xc_zhou/article/details/80952147>

## 简介

`APScheduler`的全称是`Advanced Python Scheduler`。它是一个轻量级的 Python 定时任务调度框架。**APScheduler 支持三种调度任务：固定时间间隔，固定时间点（日期），Linux 下的 Crontab 命令**。同时，它还支持异步执行、后台执行调度任务。

## 安装

pip install APScheduler

## 使用步骤

APScheduler 使用起来还算是比较简单。运行一个调度任务只需要以下三部曲：

1. 新建一个 schedulers (调度器) 。
2. 添加一个调度任务(job stores)。
3. 运行调度任务。

下面是执行每 2 秒报时的简单示例代码：

In [3]:
import datetime
import time
from apscheduler.schedulers.background import BackgroundScheduler


def timedTask():
    print('task: {}'.format(time.strftime("%d.%m.%Y %H:%M:%S")))


if __name__ == '__main__':
    # 创建后台执行的 schedulers
    scheduler = BackgroundScheduler()
    # 添加调度任务
    # 调度方法为 timedTask，触发器选择 interval(间隔性)，间隔时长为 2 秒
    scheduler.add_job(timedTask, 'interval', seconds=2)
    # 启动调度任务
    scheduler.start()

    while True:
        print('main: {}'.format(time.strftime("%d.%m.%Y %H:%M:%S")))
        time.sleep(5)

main: 11.04.2020 14:09:34
task 11.04.2020 14:09:35
task: 11.04.2020 14:09:36
task 11.04.2020 14:09:37
task: 11.04.2020 14:09:38
main: 11.04.2020 14:09:39
task 11.04.2020 14:09:39
task: 11.04.2020 14:09:40
task 11.04.2020 14:09:41
task: 11.04.2020 14:09:42
task 11.04.2020 14:09:43
main: 11.04.2020 14:09:44task: 11.04.2020 14:09:44

task 11.04.2020 14:09:45
task: 11.04.2020 14:09:46
task 11.04.2020 14:09:47
task: 11.04.2020 14:09:48


KeyboardInterrupt: 

## 基础组件

APScheduler 有四种组件，分别是：调度器(scheduler)，作业存储(job store)，触发器(trigger)，执行器(executor)。

- schedulers（调度器）  
它是任务调度器，属于控制器角色。它配置作业存储器和执行器可以在调度器中完成，例如添加、修改和移除作业。

- triggers（触发器）  
描述调度任务被触发的条件。不过触发器完全是无状态的。

- job stores（作业存储器）  
任务持久化仓库，默认保存任务在内存中，也可将任务保存都各种数据库中，任务中的数据序列化后保存到持久化数据库，从数据库加载后又反序列化。

- executors（执行器）  
负责处理作业的运行，它们通常通过在作业中提交指定的可调用对象到一个线程或者进城池来进行。当作业完成时，执行器将会通知调度器。

---

### schedulers（调度器）
我个人觉得 APScheduler 非常好用的原因。它提供 7 种调度器，能够满足我们各种场景的需要。例如：后台执行某个操作，异步执行操作等。调度器分别是：

- `BlockingScheduler` : 调度器在当前进程的主线程中运行，也就是会阻塞当前线程。
- `BackgroundScheduler` : 调度器在后台线程中运行，不会阻塞当前线程。
- `AsyncIOScheduler` : 结合 asyncio 模块（一个异步框架）一起使用。
- `GeventScheduler` : 程序中使用 gevent（高性能的Python并发框架）作为IO模型，和 GeventExecutor 配合使用。
- `TornadoScheduler` : 程序中使用 Tornado（一个web框架）的IO模型，用 ioloop.add_timeout 完成定时唤醒。
- `TwistedSchedule`r : 配合 TwistedExecutor，用 reactor.callLater 完成定时唤醒。
- `QtScheduler` : 你的应用是一个 Qt 应用，需使用QTimer完成定时唤醒。

### triggers（触发器）

APScheduler 有三种内建的 trigger:

1. date 触发器

date 是最基本的一种调度，作业任务只会执行一次。它表示特定的时间点触发。它的参数如下：

|参数|说明  |
|:----|:-------|
|run_date (datetime 或 str)	|作业的运行日期或时间
|timezone (datetime.tzinfo 或 str)|	指定时区

date 触发器使用示例如下：

In [1]:
from datetime import datetime
from datetime import date
from apscheduler.schedulers.background import BackgroundScheduler


def job_func(text):
    print(text)

    
scheduler = BackgroundScheduler()
# 在 2020-4-11时刻运行一次 job_func 方法
scheduler .add_job(job_func, 'date', run_date=date(2020, 4, 11), args=['text'])
# 在 2020-4-11 14:00:00 时刻运行一次 job_func 方法
scheduler .add_job(job_func, 'date', run_date=datetime(2020, 4, 11, 14,22, 0), args=['text'])
# 在 2020-4-11 14:00:01 时刻运行一次 job_func 方法
scheduler .add_job(job_func, 'date', run_date='2020-04-11 14:22:01', args=['text'])

scheduler.start()